In [1]:
import functools
print(dir(functools))

['RLock', 'WRAPPER_ASSIGNMENTS', 'WRAPPER_UPDATES', '_CacheInfo', '_HashedSeq', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_c3_merge', '_c3_mro', '_compose_mro', '_convert', '_find_impl', '_ge_from_gt', '_ge_from_le', '_ge_from_lt', '_gt_from_ge', '_gt_from_le', '_gt_from_lt', '_le_from_ge', '_le_from_gt', '_le_from_lt', '_lru_cache_wrapper', '_lt_from_ge', '_lt_from_gt', '_lt_from_le', '_make_key', 'cmp_to_key', 'get_cache_token', 'lru_cache', 'namedtuple', 'partial', 'partialmethod', 'recursive_repr', 'reduce', 'singledispatch', 'total_ordering', 'update_wrapper', 'wraps']


##### partial

partial function creates partial function application from another function. it is used to bind values to some of the function's arguments and produce a callable without the already defined arguments

In [3]:
from functools import partial
def f(a, b, c, x):
    return 1000*a + 100*b + 10*c + x
g = partial(f, 1, 1)
print(g(2,3))


1123


When g is created, f, which takes four arguments(a, b, c, x), is also partially evaluated for the first three arguments, a, b, c,. Evaluation of f is completed when g is called, g(2), which passes the fourth argument to f.

One way to think of partial is a shift register; pushing in one argument at the time into some function.

partial comes handy for cases where data is coming in as stream and we cannot pass more than one argument

##### cmp_to_key

Python changed its sorting methods to accept a key function. Those functions take a value and return a key which is used to sort the arrays.

Old comparison functions used to take two values and return -1, 0 or +1 if the first argument is small, equal or greater than the second argument respectively. 

This is incompatible to the new key-function.
That's where functools.cmp_to_key comes in:

In [4]:
import functools
import locale
sorted(["A", "S", "F", "D"], key=functools.cmp_to_key(locale.strcoll))

['A', 'D', 'F', 'S']

##### lru_cache(maxsize=128, typed=False)

The @lru_cache decorator can be used wrap an expensive, computationally-intensive function with a Least Recently Used cache.
This allows function calls to be memoized, so that future calls with the same parameters can return instantly instead of having to be recomputed.

Decorator to wrap a function with a memoizing callable that saves up to the maxsize most recent calls. It can save time when an expensive or I/O bound function is periodically called with the same arguments.

Since a dictionary is used to cache results, the positional and keyword arguments to the function must be hashable.

Distinct argument patterns may be considered to be distinct calls with separate cache entries. For example, f(a=1, b=2) and f(b=2, a=1) differ in their keyword argument order and may have two separate cache entries.

If maxsize is set to None, the LRU feature is disabled and the cache can grow without bound. The LRU feature performs best when maxsize is a power-of-two.

If typed is set to true, function arguments of different types will be cached separately. For example, f(3) and f(3.0) will be treated as distinct calls with distinct results.

To help measure the effectiveness of the cache and tune the maxsize parameter, the wrapped function is instrumented with a cache_info() function that returns a named tuple showing hits, misses, maxsize and currsize. In a multi-threaded environment, the hits and misses are approximate.

The decorator also provides a cache_clear() function for clearing or invalidating the cache.

The original underlying function is accessible through the __wrapped__ attribute. This is useful for introspection, for bypassing the cache, or for rewrapping the function with a different cache.

An LRU (least recently used) cache works best when the most recent calls are the best predictors of upcoming calls (for example, the most popular articles on a news server tend to change each day). The cache’s size limit assures that the cache does not grow without bound on long-running processes such as web servers.

In general, the LRU cache should only be used when you want to reuse previously computed values. Accordingly, it doesn’t make sense to cache functions with side-effects, functions that need to create distinct mutable objects on each call, or impure functions such as time() or random().

In [11]:
import functools

@functools.lru_cache(maxsize=None) # Boundless cache
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

fibonacci(10)

55

In the example above, the value of fibonacci(3) is only calculated once, whereas if fibonacci didn't have an LRU cache, fibonacci(3) would have been computed upwards of 230 times. Hence, @lru_cache is especially great for
recursive functions or dynamic programming, where an expensive function could be called multiple times with the same exact parameters

##### total_ordering

When we want to create an orderable class, normally we need to define the methods __eq()__, __lt__(), __le__(), __gt__() and __ge__().

The total_ordering decorator, applied to a class, permits the definition of __eq__() and only one between __lt__(), __le__(), __gt__() and __ge__(), and still allow all the ordering operations on the class

In [12]:
import functools

@functools.total_ordering
class Employee:
    def __eq__(self, other):
        return ((self.surname, self.name) == (other.surname, other.name))
    def __lt__(self, other):
        return ((self.surname, self.name) < (other.surname, other.name))

##### functools.reduce(function, iterable[, initializer])

Apply function of two arguments cumulatively to the items of sequence, from left to right, so as to reduce the sequence to a single value. 

For example, 

reduce(lambda x, y: x+y, [1, 2, 3, 4, 5]) 

calculates ((((1+2)+3)+4)+5). 

The left argument, x, is the accumulated value and the right argument, y, is the update value from the sequence. If the optional initializer is present, it is placed before the items of the sequence in the calculation, and serves as a default when the sequence is empty. If initializer is not given and sequence contains only one item, the first item is returned.

In [22]:
from functools import reduce
def factorial(n):
    return reduce(lambda a, b: (a*b), range(1, n+1))


print(factorial(10))
print(factorial(8))

3628800
40320


##### functools.singledispatch

Transform a function into single-dispatch generic function

to define a generic function, decorate it with @singledispatch decorator.
Note that dispatch happens on the type of first argument

In [23]:
from functools import singledispatch
@singledispatch
def fun(arg, verbose = False):
    if verbose:
        print("Let me just say", end = " ")
    print(arg)

To add overloaded implementations to the function, use the register() attribute of the generic function. It is a decorator. For functions annotated with types, the decorator will infer the type of the first argument automatically:

In [24]:
@fun.register
def _(arg: int, verbose=False):
    if verbose:
        print("Strength in numbers, eh?", end=" ")
    print(arg)

@fun.register
def _(arg: list, verbose=False):
    if verbose:
        print("Enumerate this:")
        for i, elem in enumerate(arg):
            print(i, elem)

For code which doesn’t use type annotations, the appropriate type argument can be passed explicitly to the decorator itself:

In [25]:
@fun.register(complex)
def _(arg, verbose=False):
    if verbose:
        print("Better than complicated", end=" ")
    print(arg.real, arg.imag)